In [2]:
samples_tokenized2 = np.load(f'tokenized_subset_data/train/ADA_1min.npy')

In [3]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', progress=False)

In [11]:
l = list(samples_tokenized2[0])

In [13]:
tokenizer.decode(samples_tokenized2[0])

'crypt3_1Thu00:18O0.32408H0.32422L0.32408C0.32422V600;crypt3_1Thu01:04O0.31669H0.31669L0.31669C0.31669V300;crypt3_1Thu01:07O0.31648H0.31648L0.31648C0.31648V400;crypt3_1Thu01:13O0.31639H0.31639L0.31639C0.31639V7000;crypt3_1Thu01:41O0.3154H0.3154L0.315C0.315V600;crypt3_1Thu01:53O0.32423H0.32425L0.32423C0.32425V1600;crypt3_1Thu02:10O0.31244H0.31244L0.31244C0.31244V300;crypt3_1Thu02:20O0.31138H0.31138L0.31138C0.31138V200;crypt3_1Thu02:21O0.31138H0.31138L0.31138C0.31138V600;crypt3_1Thu02:22O0.31134H0.31134L0.3105C0.3105V900;crypt3_1Thu02:23O0.315H0.315L0.315C0.315V100;crypt3_1Thu02:24O0.31H0.31L0.31C0.31V300;crypt3_1Thu02:30O0.3065H0.3065L0.3065C0.3065V100;crypt3_1Thu02:32O0.3065H0.3065L0.3065C0.3065V300;crypt3_1Thu02:36O0.305H0.305L0.305C0.305V100;crypt3_1Thu02:38O0.303H0.303L0.303C0.303V100;crypt3_1Thu02:43O0.31563H0.31563L0.31563C0.31563V600;crypt3_1Thu02:47O0.3156H0.3156L0.3156C0.3156V500;crypt3_1Thu03:05O0.31517H0.31517L0.31517C0.31517V1300;crypt3_1Thu03:37O0.3157H0.3157L0.3157C0.3157V

In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
from zipfile import ZipFile
from os import listdir
from os.path import isfile, join
import datetime
import time
import os
import numpy as np
import tensorflow as tf
from pathlib import Path
from tokenizers import Tokenizer, models, trainers
from os import listdir
from os.path import isfile, join

In [308]:
path = 'data/firstratedata/'
zipfiles = [f'crypto-active_1min_1nfzd.zip', f'crypto-active_5min_ziyhm.zip', 
            f'crypto-active_30min_wnk5g.zip', 
            f'crypto-active_1hour_9tsmq.zip']
dayOfWeek={0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
samples_tokenized = set([])
for zipfile in zipfiles:
    zf = ZipFile(f'{path+zipfile}') 
    data_files = zf.filelist
    pbar = tqdm(enumerate(data_files), total=len(data_files))
    for idx, file in pbar:
        if file.filename.startswith('USDC') or file.filename.startswith('USDT') or file.filename.startswith('BTC-EUR') or file.filename.startswith('ETH-BTC') or file.filename.startswith('UST'):
            continue
        pbar.set_postfix(file=file.filename)
        df = pd.read_csv(zf.open(file), names=['datetime', 'open', 'high', 'low', 'close', 'volume'])
        df['datetime'] = pd.to_datetime(df['datetime'])
        df['weekday'] = df['datetime'].dt.dayofweek.map(dayOfWeek)
        grid = (df['datetime'].dt.year == 2021) & (df['datetime'].dt.month == 7)
        df = df.round(6)
        
        df_train = df[~grid]
        df_val = df[grid]
        
        coin = file.filename.split('_')[0]
        tokens = ';'.join([f"crypt{row['datetime'].month}_{row['datetime'].day}{row['weekday']}{row['datetime'].time().isoformat()[:5]}O{row['open']}H{row['high']}L{row['low']}C{row['close']}V{int(row['volume'])}" for idx, row in df_train.iterrows()])
        
        text_file = open(f"strings/train/{file.filename[:-4]}_string.txt", "w")
        n = text_file.write(tokens)
        text_file.close()  
        
#         tokens = tokenizer.encode(tokens)
#         samples_tokenized.update(set(tokens))
        
        tokens = ';'.join([f"crypt{row['datetime'].month}_{row['datetime'].day}{row['weekday']}{row['datetime'].time().isoformat()[:5]}O{row['open']}H{row['high']}L{row['low']}C{row['close']}V{int(row['volume'])}" for idx, row in df_val.iterrows()])
        text_file = open(f"strings/val/{file.filename[:-4]}_string.txt", "w")
        n = text_file.write(tokens)
        text_file.close()  

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [03:03<00:00,  3.46s/it, file=ZRX_1hour.txt]


[<ZipInfo filename='ADA_1hour.txt' compress_type=deflate file_size=1612119 compress_size=358521>,
 <ZipInfo filename='BAT_1hour.txt' compress_type=deflate file_size=1472364 compress_size=347390>,
 <ZipInfo filename='BCH_1hour.txt' compress_type=deflate file_size=1847027 compress_size=602358>,
 <ZipInfo filename='BNT_1hour.txt' compress_type=deflate file_size=1697281 compress_size=442139>,
 <ZipInfo filename='BSV_1hour.txt' compress_type=deflate file_size=1360973 compress_size=441428>,
 <ZipInfo filename='BTC-EUR_1hour.txt' compress_type=deflate file_size=2150627 compress_size=715905>,
 <ZipInfo filename='BTC_1hour.txt' compress_type=deflate file_size=4192637 compress_size=1304860>,
 <ZipInfo filename='BTG_1hour.txt' compress_type=deflate file_size=1762009 compress_size=537338>,
 <ZipInfo filename='CVC_1hour.txt' compress_type=deflate file_size=653389 compress_size=135521>,
 <ZipInfo filename='DAI_1hour.txt' compress_type=deflate file_size=1235696 compress_size=256620>,
 <ZipInfo filena

In [309]:
len(samples_tokenized)

0

In [223]:
new_words = [
    f"{i}" for i in range(10)
] + [
    f"{i}{j}" for i in range(10) for j in range(10)
] + [
    f"{i}{j}{k}" for i in range(10) for j in range(10) for k in range(10)
] + [
    'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun',
    'O', 'H', 'L', 'C',
    'crypt', ';', '.',  
]

new_vocab = {}
for i, w in enumerate(new_words):  # for name, age in dictionary.iteritems():  (for Python 2.x)
    new_vocab[w] = i

In [326]:
merges = [str(i) for i in range(99)]
# for i in range(10):
#     for j in range(10):
#         for k in range(10):
#             merges.append(f'{i}{j} {k}\n')
# for i in range(10):
#     for j in range(10):
#         for k in range(10):
#             merges.append(f'{i} {j}{k}\n')
# for i in range(10):
#     for j in range(10):
#         merges.append(f'{i} {j}\n')

f = open('merges.txt', 'r', encoding="utf8")
lines = f.readlines()
lines[1:]
f.close()
with open('my_tokenizer/merges.txt', 'w') as filehandle:
    for line in lines:
        t = line.replace(' ', '').replace('\n', '')
        if len(t) <= 3 and t.isdigit():
            if any([m in line for m in merges]): 
                filehandle.write(line)
                print(line)

0 0

0 1

00 0

1 0

1 2

5 0

1 9

1 1

2 0

2 01

3 0

1 5

1 4

1 6

1 3

2 5

1 8

1 7

2 4

8 0

4 0

2 2

6 0

2 3

2 9

2 7

2 6

2 8

9 9

3 3

7 0

2 00

4 5

3 5

6 4

7 5

2 1

3 8

4 4

3 6

3 2

3 9

3 4

0 5

3 7

4 8

6 6

5 5

4 7

0 8

4 9

0 9

6 5

0 7

0 2

0 4

1 00

0 3

6 8

3 1

6 7

5 9

0 6

7 7

5 8

6 9

8 8

4 6

5 7

4 3

4 2

7 8

7 9

9 0

9 5

4 1

5 6

5 4

5 00

9 8

7 6

5 2

5 3

5 1

8 6

7 4

8 9

7 2

7 3

9 6

7 1

6 3

6 2

8 5

6 1

9 7

8 4

8 7

9 4

9 2

8 3

9 3

3 00

9 1

8 2

8 1

4 00

8 00

6 00

00 1

1 50

1 01

2 50

7 00

12 3

12 0

00 3

1 10

12 5

25 6

4 29

12 8

9 00

1 30

10 5

25 5

10 4

16 0

10 9

16 8

1 12

3 50

14 0

10 3

18 0

3 60

11 5

10 2

7 50

10 8

10 6

10 7

24 0

11 1

1 19

11 4

11 3

12 7

11 8

13 3

99 9

13 5

17 0

17 5

12 4

19 2

6 50

2 20

11 7

15 2

4 50

2 25

14 8

11 6

12 2

14 4

15 7

20 4

12 9

15 5

14 5

26 4

3 01

20 2

19 9

2 30

12 1

12 6

14 9

3 20

15 9

13 7

19 0

13

In [323]:
new_vocab = {}
i = 0
for k, v in tokenizer.vocab.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    if v in samples_tokenized:
        new_vocab[k] = i
        i += 1

In [324]:
len(new_vocab)

0

In [325]:
import json
json.dump(new_vocab, open('my_tokenizer/vocab_mine.json', 'w'))

In [293]:
tokenizer_new = GPT2TokenizerFast('my_tokenizer/vocab_mine.json', 'my_tokenizer/merges.txt')

In [307]:
t = "110"
tokenizer_new.encode(t)

[220]

In [306]:
tokenizer_new.convert_ids_to_tokens(tokenizer_new.encode(t))

['0', '010']

In [125]:
tokenizer.save_vocabulary('./')

('./vocab.json', './merges.txt')

In [310]:
tokenizer_new.vocab_size

1124

In [311]:
train_done_files = [f[:-4] for f in listdir('tokenized_data/train/')]
val_done_files = [f[:-4] for f in listdir('tokenized_data/val/')]

train_files = [f for f in listdir('strings/train/') if f[:-11] not in train_done_files]
val_files = [f for f in listdir('strings/val/') if f[:-11] not in val_done_files]

In [312]:
for frame in ['_1min', '5min', '_30min', '_1hour']:
    print(frame)
    for file in train_files:
        if frame in file:
            samples_tokenized2 = []
            print(file)
            f = Path('strings/train/'+file).read_text()
            tokens = tokenizer_new.encode(f)
            for i in range(0, len(tokens), 2048):
                samples_tokenized2.append(tokens[i:i+2048])
            del samples_tokenized2[-1]
            
            samples_tokenized2 = np.array(samples_tokenized2)
            np.save(f'tokenized_data/train/{file[:-11]}', samples_tokenized2)

_1min
ADA_1min_string.txt
BAT_1min_string.txt
BCH_1min_string.txt
BNT_1min_string.txt
BSV_1min_string.txt
BTC_1min_string.txt
BTG_1min_string.txt
CVC_1min_string.txt
DAI_1min_string.txt
DASH_1min_string.txt
DCR_1min_string.txt
DOGE_1min_string.txt
EOS_1min_string.txt
ETC_1min_string.txt
ETH_1min_string.txt
FUN_1min_string.txt
HT_1min_string.txt
ICX_1min_string.txt
IOST_1min_string.txt
KNC_1min_string.txt
LINK_1min_string.txt
LRC_1min_string.txt
LSK_1min_string.txt
LTC_1min_string.txt
MAID_1min_string.txt
MANA_1min_string.txt
MKR_1min_string.txt
NEO_1min_string.txt
OMG_1min_string.txt
ONT_1min_string.txt
PAX_1min_string.txt
QTUM_1min_string.txt
REP_1min_string.txt
SC_1min_string.txt
SNT_1min_string.txt
TRX_1min_string.txt
UTK_1min_string.txt
VET_1min_string.txt
WAVES_1min_string.txt
XEM_1min_string.txt
XLM_1min_string.txt
XMR_1min_string.txt
XRP_1min_string.txt
XTZ_1min_string.txt
XVG_1min_string.txt
ZEC_1min_string.txt
ZIL_1min_string.txt
ZRX_1min_string.txt
5min
ADA_5min_string.txt
BA

In [313]:
for frame in ['_1min', '5min', '_30min', '_1hour']:
    print(frame)
    for file in val_files:
        if frame in file:
            samples_tokenized2 = []
            print(file)
            f = Path('strings/val/'+file).read_text()
            tokens = tokenizer_new.encode(f)
            for i in range(0, len(tokens), 2048):
                samples_tokenized2.append(tokens[i:i+2048])
            if len(samples_tokenized2) > 0:
                del samples_tokenized2[-1]
            
            samples_tokenized2 = np.array(samples_tokenized2)
            np.save(f'tokenized_data/val/{file[:-11]}', samples_tokenized2)

_1min
ADA_1min_string.txt
BAT_1min_string.txt
BCH_1min_string.txt
BNT_1min_string.txt
BSV_1min_string.txt
BTC_1min_string.txt
BTG_1min_string.txt
CVC_1min_string.txt
DAI_1min_string.txt
DASH_1min_string.txt
DCR_1min_string.txt
DOGE_1min_string.txt
EOS_1min_string.txt
ETC_1min_string.txt
ETH_1min_string.txt
FUN_1min_string.txt
HT_1min_string.txt
ICX_1min_string.txt
IOST_1min_string.txt
KNC_1min_string.txt
LINK_1min_string.txt
LRC_1min_string.txt
LSK_1min_string.txt
LTC_1min_string.txt
MAID_1min_string.txt
MANA_1min_string.txt
MKR_1min_string.txt
NEO_1min_string.txt
OMG_1min_string.txt
ONT_1min_string.txt
PAX_1min_string.txt
QTUM_1min_string.txt
REP_1min_string.txt
SC_1min_string.txt
SNT_1min_string.txt
TRX_1min_string.txt
UTK_1min_string.txt
VET_1min_string.txt
WAVES_1min_string.txt
XEM_1min_string.txt
XLM_1min_string.txt
XMR_1min_string.txt
XRP_1min_string.txt
XTZ_1min_string.txt
XVG_1min_string.txt
ZEC_1min_string.txt
ZIL_1min_string.txt
ZRX_1min_string.txt
5min
ADA_5min_string.txt
BA

In [314]:
def _int64_feature(value):
    """
    Returns an int64_list from a bool / enum / int / uint.
    """
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def write_to_file(writer, data):
    """
    writes data to tfrecord file
    """
    feature = {
        "text": _int64_feature(data)
    }
    tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(tf_example.SerializeToString())
    
    
def split_list(l, n):
    # splits list/string into n size chunks
    return [l[i:i + n] for i in range(0, len(l), n)]


def write_files(files, files_per, output_dir, out_name, start_no, write_remainder=True, process_no=None):
    # writes a list of files to .tfrecords
    chunks = split_list(files, files_per)

    if len(chunks[-1]) != files_per and not write_remainder:  # pop the last file if it's length != files per
        remainder = chunks.pop(-1)
    else:
        remainder = None  # assuming files = remainder from an old chunk here
        files_per = len(chunks[-1])

    for files in chunks:
        fp = f"{output_dir}/{out_name}_{start_no}"
        if process_no is not None:
            fp += f"_{process_no}"
        fp += f"_{files_per}"  # add number of files in tfrecord to end of fp
        fp += ".tfrecords"
        with tf.io.TFRecordWriter(fp) as writer:
            for f in files:
                write_to_file(writer, f)
        start_no += 1
    return start_no, remainder


def create_tfrecords(tokenized_files_array, output_dir, out_name):
    files_per = 100000
    _tfrecord_count, remainder = write_files(tokenized_files_array, files_per=files_per,
                                                         output_dir=output_dir, out_name=out_name,
                                                         start_no=0, process_no=None)
    
@tf.function
def decode_fn(record_bytes):
  return tf.io.parse_single_example(
      # Data
      record_bytes,

      # Schema
      {"x": tf.io.FixedLenFeature([], dtype=tf.float32),
       "y": tf.io.FixedLenFeature([], dtype=tf.float32)}
  )

In [315]:
samples_tokenized3 = np.concatenate([np.load(f'tokenized_data/train/{file[:-11]}.npy') for file in train_files])
results = create_tfrecords(samples_tokenized3, 'records_scratch', 'crypto_train')

In [322]:
samples_tokenized3 = np.concatenate([np.load(f'tokenized_data/val/{file[:-11]}.npy') for file in val_files if np.load(f'tokenized_data/val/{file[:-11]}.npy').shape[0] > 0])
results = create_tfrecords(samples_tokenized3, 'records_scratch', 'crypto_val')